In [2]:
import pandas as pd

In [18]:
df_ngan = pd.read_csv('df_final_ngan.csv')
df_ngan['cleaned_name'] = df_ngan['name'].str.lower()
df_ngan["cleaned_name"].replace({"vision '74 inc.": "vision nursing home", "finlandia hoivakoti nursing home limited": "finlandia hoivakoti nursing home", "pinecrest nursing home - bobcaygeon": "pinecrest nursing home (bobcaygeon)", "residence saint- louis": "residence saint-louis", "st patrick's home": "st. patrick's home"}, inplace=True)
df_ngan.head()

,name,address,city and postal code,LHIN,licensee,management,home type,beds,short stay,residents council,family council,accreditation,additional_info,city,postal code,number of beds,total_inspections,5y_inspections,cleaned_name
0,AFTON PARK PLACE LONG TERM CARE COMMUNITY,1200 Afton Drive,"Sarnia, N7S6L6",Erie St. Clair,S & R Nursing Homes Ltd.,NaN,For-Profit,Home with approximately 128 beds,No,Yes,Yes,Yes,NaN,Sarnia,N7S6L6,128.0,67.0,34.0,afton park place long term care community
1,"ALBRIGHT GARDENS HOMES, INCORPORATED",5050 Hillside Drive,"Beamsville, L0R1B2",Hamilton Niagara Haldimand Brant (Hnhb),"Albright Gardens Homes, Incorporated",NaN,Non-Profit,Home with approximately 231 beds,No,Yes,Yes,No,NaN,Beamsville,L0R1B2,231.0,39.0,25.0,"albright gardens homes, incorporated"
2,ALEXANDER PLACE,329 Parkside Drive P. O. Box 50,"Waterdown, L0R2H0",Hamilton Niagara Haldimand Brant (Hnhb),Waterdown Long Term Care Centre Inc.,NaN,For-Profit,Home with approximately 128 beds,Yes,Yes,Yes,Yes,NaN,Waterdown,L0R2H0,128.0,28.0,17.0,alexander place
3,ALGOMA DISTRICT HOMES FOR THE AGED (ALGOMA MANOR),135 Dawson Street,"Thessalon, P0R1L0",North East,Board Of Management For The District Of Algoma,NaN,NaN,Home with approximately 108 beds,Yes,Yes,Yes,No,Home Closed on 11/01/2011,Thessalon,P0R1L0,108.0,5.0,0.0,algoma district homes for the aged (algoma manor)
4,ALGOMA MANOR NURSING HOME,145 Dawson Street,"Thessalon, P0R1L0",North East,Algoma Manor Nursing Home,NaN,NaN,Home with approximately 96 beds,Yes,Yes,Yes,Yes,NaN,Thessalon,P0R1L0,96.0,23.0,14.0,algoma manor nursing home


In [19]:
df_kt = pd.read_csv('merged_ltc.csv')
df_kt['cleaned_name'] = df_kt['LTC Home'].str.lower()
df_kt.head()

,Unnamed: 0,LTC Home,City,Beds,Confirmed Resident Cases,Resident Deaths,Confirmed Staff Cases,Status,cleaned_name
0,0,Almonte Country Haven,Almonte,82,<5,28,13,Active,almonte country haven
1,1,Altamont Care Community,Scarborough,159,72,46,60,Active,altamont care community
2,2,Anson Place Care Centre,Hagersville,61,28,23,29,Active,anson place care centre
3,3,Arbour Creek Long-Term Care Centre,Hamilton,129,0,0,<5,Active,arbour creek long-term care centre
4,4,Avalon Retirement Centre,Orangeville,137,0,0,<5,Active,avalon retirement centre


In [20]:
a = set(df_ngan['cleaned_name'])
b = set(df_kt['cleaned_name'])
def returnNotMatches(a, b):
    return [[x for x in b if x not in a]]
returnNotMatches(a, b)

[[]]

In [21]:
df_merged = pd.merge(left=df_ngan, right=df_kt, how='outer', left_on='cleaned_name', right_on='cleaned_name')

In [22]:
df_merged.to_csv(r'df_ltc_final.csv', index = False)

In [27]:
df_on = pd.read_csv('odhf_ontario.csv')
df_on.head()

import unicodedata

df_on['cleaned_name'] = df_on['facility_name'].apply(lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode())
df_on['cleaned_name'] = df_on['cleaned_name'].str.lower()

In [33]:
a = set(df_on['cleaned_name'])
b = set(df_ngan['cleaned_name'])
def returnNotMatches(a, b):
    return [x for x in b if x not in a]
missing_odhf = returnNotMatches(a, b)

In [39]:
len(missing_odhf)


36

In [38]:
missing_odhf

['north renfrew long-term care services',
 'georgian bay general hospital - one north',
 'élisabeth-bruyère residence',
 'north shore health network – ltc unit',
 'bon air long term care residence',
 'caressant care cobden',
 'the meadows',
 'bonnie brae health care centre',
 'lady isabelle nursing home',
 'malden park continuing care centre',
 'marianhill - marguerite centre',
 'caressant care harriston',
 'great northern nursing centre',
 "st. joseph's mother house (martha wing)",
 "st. joseph's health care, london - mount hope centre for long term care - marian villa",
 'bella senior care residences inc.',
 'lancaster long term care residence',
 'niagara long term care residence',
 "st. joseph's villa, dundas",
 'strathcona long term care',
 'extendicare tendercare',
 'residence saint-louis',
 'dawson court',
 'huntsville district memorial hospital',
 'thunder bay interim long-term care centre',
 'algoma district homes for the aged (algoma manor)',
 'picton manor nursing home',
 'ch